# 코로나 뉴스 웹 크롤링
* 구글에서, 특정 기간, 특정 키워드 검색
* 뉴스 타이틀만 가져오기
* wordcloud로 시각화 하기

## 2020년 3월

## selenium, bs 이용

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome('chromedriver_94')
url = 'https://www.google.com/'
driver.get(url)
time.sleep(0.5)

In [3]:
# 검색창 xpath : /html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input
# 검색 버튼 xpath : /html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]
search_window = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_button = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]')

In [4]:
search_window.send_keys('집단감염')
search_button.click()
time.sleep(1)

In [5]:
# 뉴스 탭 : //*[@id="hdtb-msb"]/div[1]/div/div[3]/a
news_tab = driver.find_element_by_xpath('//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')
news_tab.click()
time.sleep(1)

### 기간 지정

In [6]:
# 도구 : //*[@id="hdtb-tls"]
tool = driver.find_element_by_xpath('//*[@id="hdtb-tls"]')
tool.click()

In [7]:
# 최근항목 : //*[@id="hdtbMenus"]/span[2]/g-popup/div[1]/div/div
recent = driver.find_element_by_xpath('//*[@id="hdtbMenus"]/span[2]/g-popup/div[1]/div/div')
recent.click()

In [8]:
# 기간 설정 : //*[@id="lb"]/div/g-menu/g-menu-item[8]/div/div/span
period_set = driver.find_element_by_xpath('//*[@id="lb"]/div/g-menu/g-menu-item[8]/div/div/span')
period_set.click()

In [9]:
# 시작일 입력창(월/일/년) : //*[@id="OouJcb"]
# 종료일 입력창(월/일/년) : //*[@id="rzG2be"]
# 실행 버튼 : //*[@id="T3kYXe"]/g-button
start_date = driver.find_element_by_xpath('//*[@id="OouJcb"]')
end_date = driver.find_element_by_xpath('//*[@id="rzG2be"]')
launch_button = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')

start_date.send_keys('3/1/2020')
end_date.send_keys('3/31/2020')
time.sleep(0.5)
launch_button.click()

In [10]:
# 페이지 xpath
# 2페이지 버튼 : //*[@id="xjs"]/table/tbody/tr/td[3]/a
# 3페이지 버튼 : //*[@id="xjs"]/table/tbody/tr/td[4]/a
# 4페이지 버튼 : //*[@id="xjs"]/table/tbody/tr/td[5]/a
# 5페이지 버튼 : //*[@id="xjs"]/table/tbody/tr/td[6]/a

In [11]:
# xpath는 오류가 많음.
# tag로 해보자.

In [12]:
# 1페이지 뉴스 타이틀(tag, 규칙 찾기)
# 1 : div, class='mCBkyc JQe2Ld nDgy9d', role="heading"
# 2:  div, class='mCBkyc JQe2Ld nDgy9d', role="heading"
# 3 : div, class="mCBkyc JQe2Ld nDgy9d", role="heading"

In [13]:
# 2페이지 뉴스 타이틀(tag, 규칙 찾기)
# 1: div, class="mCBkyc JQe2Ld nDgy9d", role="heading"
# 2 : div, class="mCBkyc JQe2Ld nDgy9d", role="heading"
# 3 : div, class="mCBkyc JQe2Ld nDgy9d", role="heading"

In [14]:
page = driver.page_source
soup = BeautifulSoup(page, 'lxml')
soup.title

<title>집단감염 - Google 검색</title>

In [15]:
test_news_titles = soup.find_all("div", class_='mCBkyc JQe2Ld nDgy9d')
test_news_titles[0].text

"콜센터 확진: 박원순 '콜센터 관련 확진자 93명… 건물 다른 사람들도 조사 중'"

In [16]:
for one in test_news_titles:
    print(one.text)

콜센터 확진: 박원순 '콜센터 관련 확진자 93명… 건물 다른 사람들도 조사 중'
“또 집단감염 나올라” 금융권 콜센터 실태조사…재택근무 고려도
'서울 최대 집단감염' 구로구 콜센터, 확진자 급증…안양·광명·김포 등서 추가
[포토] 성남 '은혜의강' 교회 집단 감염 발생
[2보] 수도권 또 집단감염…성남 은혜의강 교회 하루새 40명 확진
"은혜의강 교회, 코로나 잡는다며 신도 입에 소금물 분사... 오히려 집단 감염 확산" - 조선일보
코로나19: 대구 한 요양병원서 74명 집단감염
대구 정신병원 또 집단감염…뒷북 행정 '예견된 사태'
[코로나19 집단감염] 신도림동 코리아빌딩 콜센터 최소 32명
정부, 요양병원 집단감염시 '손해배상' 청구···의료계 "이제 와 ...


In [18]:
all_news_titles = []

# 1페이지 크롤링
time.sleep(1)

page = driver.page_source
soup = BeautifulSoup(page, 'lxml')

news_titles = soup.find_all("div", class_='mCBkyc JQe2Ld nDgy9d')
for one in news_titles:
    all_news_titles.append(one.text)


# 2-15페이지 크롤링
for p in range(3, 17):
    move_page = driver.find_element_by_xpath(f'//*[@id="xjs"]/table/tbody/tr/td[{p}]/a')
    move_page.click()
    time.sleep(1)
    
    # bs
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    news_titles = soup.find_all("div", class_='mCBkyc JQe2Ld nDgy9d')
    for one in news_titles:
        all_news_titles.append(one.text)

all_news_titles

["콜센터 확진: 박원순 '콜센터 관련 확진자 93명… 건물 다른 사람들도 조사 중'",
 '“또 집단감염 나올라” 금융권 콜센터 실태조사…재택근무 고려도',
 "'서울 최대 집단감염' 구로구 콜센터, 확진자 급증…안양·광명·김포 등서 추가",
 "[포토] 성남 '은혜의강' 교회 집단 감염 발생",
 '[2보] 수도권 또 집단감염…성남 은혜의강 교회 하루새 40명 확진',
 '"은혜의강 교회, 코로나 잡는다며 신도 입에 소금물 분사... 오히려 집단 감염 확산" - 조선일보',
 '코로나19: 대구 한 요양병원서 74명 집단감염',
 "대구 정신병원 또 집단감염…뒷북 행정 '예견된 사태'",
 '[코로나19 집단감염] 신도림동 코리아빌딩 콜센터 최소 32명',
 '정부, 요양병원 집단감염시 \'손해배상\' 청구···의료계 "이제 와 ...',
 '창녕 동전노래방 확진자 6명째…소규모 집단감염 현실화하나',
 '또 터진 병원 코로나19 집단감염…해외 유입 증가세도 여전',
 '대구 제2미주병원 133명 집단감염…청도대남병원 복사판',
 '환풍기 작동 안됐다, 확진 58명 쏟아진 제2미주병원선 무슨 일 - 중앙일보',
 '11층 벗어난 구로 콜센터 집단감염…제주까지 비상 걸렸다 - 중앙일보',
 '＂코로나19 전국서 집단감염 시 신천지 확진자 수 발생 가능＂',
 "'국민안심병원' 분당제생병원서 코로나19 집단감염 발생한 이유 ...",
 "집단감염 '줌바댄스' 강사 3명서 4차 전파까지 확인",
 "이집트 나일강 크루즈선 45명 '코로나19' 집단 감염",
 '"구로 콜센터 11층 감염률 43.5%…전체 확진자 가족 15%가 감염"',
 '"국내 코로나19, 80%가 집단감염"…서울 콜센터 최대 발생 비상',
 '국내 코로나19 감염 10명 중 8명은 집단감염',
 '[‘코로나19’ 확산 비상]정부, 집단감염 우려 사업장 ‘집중관리 지침’ 마련…PC방·노래방 ‘불특정 다수’ 고객 통제 가능할까',
 "대구 한전MCS 지사서 '코로나19' 12명 집단감염",
 "

In [19]:
# 10페이지 15개씩 => 15개
len(all_news_titles)

150

In [20]:
news = {"'2020-03": all_news_titles}
dat = pd.DataFrame(news)
dat.to_csv('2020년03월_집단감염_뉴스(구글).csv', index=False)

## 2020년 8월

In [21]:
driver = webdriver.Chrome('chromedriver_94')
url = 'https://www.google.com/'
driver.get(url)
time.sleep(0.5)
search_window = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_button = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]')
search_window.send_keys('집단감염')
search_button.click()
time.sleep(1)

In [22]:
# 뉴스 탭 : //*[@id="hdtb-msb"]/div[1]/div/div[2]/a
news_tab = driver.find_element_by_xpath('//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')
news_tab.click()
time.sleep(1)

### 기간지정

In [23]:
# 도구 : //*[@id="hdtb-tls"]
tool = driver.find_element_by_xpath('//*[@id="hdtb-tls"]')
tool.click()

In [24]:
# 최근항목 : //*[@id="hdtbMenus"]/span[2]/g-popup/div[1]/div/div
recent = driver.find_element_by_xpath('//*[@id="hdtbMenus"]/span[2]/g-popup/div[1]/div/div')
recent.click()

In [25]:
# 기간 설정 : //*[@id="lb"]/div/g-menu/g-menu-item[8]/div/div/span
period_set = driver.find_element_by_xpath('//*[@id="lb"]/div/g-menu/g-menu-item[8]/div/div/span')
period_set.click()

In [26]:
# 시작일 입력창(월/일/년) : //*[@id="OouJcb"]
# 종료일 입력창(월/일/년) : //*[@id="rzG2be"]
# 실행 버튼 : //*[@id="T3kYXe"]/g-button
start_date = driver.find_element_by_xpath('//*[@id="OouJcb"]')
end_date = driver.find_element_by_xpath('//*[@id="rzG2be"]')
launch_button = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')

start_date.send_keys('8/1/2020')
end_date.send_keys('8/31/2020')
time.sleep(0.5)
launch_button.click()

In [29]:
all_news_titles = []

# 1페이지 크롤링
time.sleep(1)

page = driver.page_source
soup = BeautifulSoup(page, 'lxml')

news_titles = soup.find_all("div", class_='mCBkyc JQe2Ld nDgy9d')
for one in news_titles:
    all_news_titles.append(one.text)


# 2-13페이지 크롤링
for p in range(3, 15):
    move_page = driver.find_element_by_xpath(f'//*[@id="xjs"]/table/tbody/tr/td[{p}]/a')
    move_page.click()
    time.sleep(1)
    
    # bs
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    news_titles = soup.find_all("div", class_='mCBkyc JQe2Ld nDgy9d')
    for one in news_titles:
        all_news_titles.append(one.text)

all_news_titles

['사랑제일교회발 집단감염 전국 75곳으로 퍼졌다...138명 늘어난 ...',
 '전광훈 사랑제일교회 집단감염 제2의 신천지 사태되나…"그때 ...',
 '서울지역 교회 집단감염 급증...8월에만 470명 확진·1∼7월의 3배 - 조선비즈',
 "집단감염 '용인 우리제일교회' 11명 추가 확진…나흘 동안 24명",
 "'코로나 재확산'에 교회 집단감염 논란 재점화…교계는 '당혹'",
 "'집단감염 폐쇄' 사랑제일교회 썰렁…전광훈 광복절 집회 독려",
 '대구 교회 집단 감염… 29명 확진자 추가 발생',
 '남대문시장 중앙상가에서도 확진자…1명 추가 감염(종합)',
 '구로구 아파트 같은 라인만 집단감염, 환기구 때문?',
 "'롯데리아발' 집단감염 경기도 확진자 5명…방역당국 비상",
 '커피숍 집단감염, 홍천 캠핑 확진자와 가까운 자리 앉았다',
 "강남 커피점 집단감염 2명 추가 누적 12명…경북까지 'n차 전파'(종합)",
 "할리스 집단감염에 커피업계 '긴장'… 코로나 예방 재정비 - 조선비즈",
 '인천 교회서 29명 집단감염…광화문집회 참가자도 포함',
 '코로나19 전방위 확산,새 집단감염 속출…사랑제일교회 총 915명',
 "감염경로 모르는 '깜깜이' 환자 33.2%…방역당국 점점 힘들어진다",
 "순천 헬스장서 12명 집단감염…'마스크 안 쓰고 운동'",
 "원주 집단감염 '일파만파'…공무원 포함 이틀 새 13명 확진",
 '코로나 신규 확진 324명···3월 신천지 집단감염 이후 최대',
 '\'교인 집단 감염\' 광주 성림침례교회 "머리 숙여 사죄"',
 '코로나19 299명 추가 확진…대구서 30명 집단감염 신천지 이후 ...',
 "교회 집단감염 'n차 전파' 확산 비상…정부, 교회 방역강화 검토(종합)",
 '신규 확진 28명…남대문시장 집단감염은 고양 교회발',
 '고양시 교회 집단감염 총 44명… 반석교회 24명, 기쁨153교회 20명',
 "'살인 진드기병' STFS 진료하던 의료진 5명 집단감염",
 "'18명 집단감염' 남

In [30]:
len(all_news_titles)

130

In [31]:
news = {"'2020-08": all_news_titles}
dat = pd.DataFrame(news)
dat.to_csv('2020년08월_집단감염_뉴스(구글).csv', index=False)